# Develop odometry
## Experiments: 2018-07-03<br>
Erik Frisk (erik.frisk@liu.se)<br>
Department of Electrical Engineering<br>
Linköping University<br>
Sweden

## Imports

In [ ]:
import sys
if not ('../python' in sys.path):
    sys.path.append('../python')

import matplotlib.pyplot as plt
from calib_util import LoadLogDataPickle, BoxOff
from calib_util import QualisysSpeed, OdoSpeed, GetCalibrationSection
import numpy as np
import pandas as pd
from scipy.optimize import minimize

datadir = '../cascar_logs/20180703/'

In [ ]:
%matplotlib 

#from pylab import rcParams
#%matplotlib inline
#rcParams['figure.figsize']=np.array(rcParams['figure.figsize'])*1.8

# Define helper functions

In [ ]:
def EvalOdometry(segment, x0, c0, c1, r, L, W):
    def SteerFunction(steer, c0, c1):
        return c0 + steer*c1
    
    odo_hat = np.zeros((len(segment), 3))
    odo_hat[0, :] = x0[0:3]

    for k in range(0, len(segment)-1):
        delta = SteerFunction(segment['steer'].values[k], c0, c1)
        ds = np.pi*2*r/10*1/(1-W/2/L*np.tan(delta))
        odo_hat[k+1, :] = odo_hat[k, :] + ds*np.array([np.cos(odo_hat[k, 2]),
                                                       np.sin(odo_hat[k, 2]),
                                                       1/L*np.tan(delta)])

    odo_hat = pd.DataFrame(
        np.hstack((segment['t'].values.reshape(-1, 1), odo_hat)),
        columns=['t', 'x', 'y', 'phi'])
    
    err = (segment[['x', 'y']].values-odo_hat[['x', 'y']].values)
    loss = 1/err.shape[0]*np.sqrt(np.sum(err[:, 0]**2 + err[:, 1]**2))    
    
    return odo_hat, loss

def EstimateOdometry_rLW(segment, x0, theta0):
    def loss_fcn(theta):
        _, l = EvalOdometry(segment, x0, theta0[0], theta0[1], *theta)
        return l
    
    th0 = theta0[2:5]  # r, L, W
    res = minimize(loss_fcn, th0, 
                   method='nelder-mead', 
                   options={'xtol':1e-4, 'disp':True, 'maxiter':2000})
    thopt = np.hstack(([c0, c1], res.x))
    return thopt, res

def EstimateOdometry_crW(segment, x0, theta0):
    def loss_fcn(theta):
        _, l = EvalOdometry(segment, x0, theta[0], theta[1], theta[2],
                            theta0[3], theta[3])
        return l
    
    th0 = np.array(theta0)[np.array([0, 1, 2, 4])]  # c0, c1, r, W
    res = minimize(loss_fcn, th0, 
                   method='nelder-mead', 
                   options={'xtol':1e-4, 'disp':True, 'maxiter':2000})
    thopt = np.hstack((res.x[[0, 1, 2]], [theta0[3], res.x[3]]))
    return thopt, res

def EstimateOdometry_all(segment, x0, theta0):
    def loss_fcn(theta):
        _, l = EvalOdometry(segment, x0, *theta)
        return l
    res = minimize(loss_fcn, theta0, 
                   method='nelder-mead', 
                   options={'xtol':1e-4, 'disp':True, 'maxiter':2000})
    return res.x, res

def DelaySteer(data, tau):
    data_dt = {}
    for ds in data.keys():
        data_dt[ds] = {'pos': data[ds]['pos'], 'odo': data[ds]['odo'].copy()}
        data_dt[ds]['odo']['steer_orig'] = data_dt[ds]['odo']['steer']
        data_dt[ds]['odo']['steer'] = np.interp(data_dt[ds]['odo']['t'], data_dt[ds]['odo']['t']+tau, data_dt[ds]['odo']['steer'])
    return data_dt

## Load data

In [ ]:
data = {}

pos, odo_l, odo_r = LoadLogDataPickle(datadir + 'cascar_201873_13_58_slow.pickle', 
                                      body='cascar', n_skip=1, 
                                      calibrate_phi=True, synchronize=True)
odo = odo_l
v_qualisys = QualisysSpeed(pos)
v_odo = OdoSpeed(odo)
data['slow'] = {'pos': pos, 'odo': odo}

pos, odo_l, odo_r = LoadLogDataPickle(datadir + 'cascar_201873_14_1_fast.pickle', 
                                      body='cascar', n_skip=1, 
                                      calibrate_phi=True, synchronize=False)
odo = odo_l
data['fast'] = {'pos': pos, 'odo': odo}
data['fast']['pos']['t'] = data['fast']['pos']['t'] - 1.9

Plot data and verify odometry/positioning synchronization

In [ ]:
plt.figure(10, clear=True)
plt.subplot(1, 2, 1)
plt.plot(data['slow']['pos']['x'], data['slow']['pos']['y'])
plt.xlabel('x [m]')
plt.ylabel('y [m]')
plt.title('Slow')
BoxOff()

plt.subplot(1, 2, 2)
plt.plot(data['slow']['pos']['t'], QualisysSpeed(data['slow']['pos']), 'b', label='v_pos')
plt.plot(data['slow']['odo']['t'], OdoSpeed(data['slow']['odo']), 'r', label='v_odo')
plt.legend()
plt.xlabel('t [s]')
plt.ylabel('speed [m/s]')
plt.title('Slow')
BoxOff()

plt.figure(11, clear=True)
plt.subplot(1, 2, 1)
plt.plot(data['fast']['pos']['x'], data['fast']['pos']['y'])
plt.xlabel('x [m]')
plt.ylabel('y [m]')
plt.title('Fast')
BoxOff()

plt.subplot(1, 2, 2)
plt.plot(data['fast']['pos']['t'], QualisysSpeed(data['fast']['pos']), 'b', label='v_pos')
plt.plot(data['fast']['odo']['t'], OdoSpeed(data['fast']['odo']), 'r', label='v_odo')
plt.legend()
plt.xlabel('t [s]')
plt.ylabel('speed [m/s]')
plt.title('Fast')
BoxOff()

In [ ]:
N = 10
d = data['fast']
odom, x0 = GetCalibrationSection(d['pos'], d['odo'], 11, 16)

plt.figure(100, clear=True)
plt.subplot(1, 2, 1)
plt.plot(odom['x'], odom['y'])
plt.plot(odom['x'][0:1], odom['y'][0:1], 'go')
plt.plot(odom['x'][::N], odom['y'][::N], 'rx')
plt.xlabel('x [m]')
plt.ylabel('y [m]')
BoxOff()

fig = plt.subplot(1, 2, 2)
ax1 = fig.axes
ax2 = ax1.twinx()
ax1.plot(odom['t'], odom['steer'], 'b', label='steer')
ax1.plot(odom['t'][::N], odom['steer'][::N], 'rx', label='steer')
ax2.plot(odom['t'], odom['vel'], 'r', label='velocity')

ax1.set_ylabel('steer [%]')
ax2.set_ylabel('velocity [%]')
#ax1.legend()
#ax2.legend()

ax1.set_xlabel('t [s]')
plt.tight_layout()

# Set initial parameter values

In [ ]:
# c0 = 0.02840204640670601
# c1 = 0.0033528640065687834
c0 = 0.0202330878136394
c1 = 0.0016996712762432785
r = 78.0/1000/2
L = 28.0/100
W = 21.0/100 
theta0 = [c0, c1, r, L, W]

# Odometry investigation

Add time delay to steer signal in odometry data

In [ ]:
tau = 275/1000
data_dt = DelaySteer(data, tau)

plt.figure(20, clear=True)
for k, ds in enumerate(data_dt.keys()):
    plt.subplot(1, 2, k + 1)
    plt.plot(data_dt[ds]['odo']['t'], data_dt[ds]['odo']['steer_orig'], 'b', label='original')
    plt.plot(data_dt[ds]['odo']['t'], data_dt[ds]['odo']['steer'], 'r', label='dt')
    plt.legend(loc='upper left')
    plt.xlabel('t [s]')
    plt.title('Data set: ' + ds)
    BoxOff()
plt.tight_layout()

## Slow speed

In [ ]:
d = data_dt['slow']
est = [15, 20]
test = [28, 35]
#test = est
odom_est, x0_est = GetCalibrationSection(d['pos'], d['odo'], est[0], est[1])
odom_test, x0_test = GetCalibrationSection(d['pos'], d['odo'], test[0], test[1])


gamma = 0.6
plt.figure(30, clear=True)
plt.plot(d['pos']['x'], d['pos']['y'], label='')
plt.plot(x0_est[0] + gamma*np.array([0, np.cos(x0_est[2])]),
         x0_est[1] + gamma*np.array([0, np.sin(x0_est[2])]), 'k', lw=2)
plt.plot(x0_test[0] + gamma*np.array([0, np.cos(x0_test[2])]),
         x0_test[1] + gamma*np.array([0, np.sin(x0_test[2])]), 'k', lw=2)

plt.plot(odom_est['x'], odom_est['y'], 'b', lw=4, label='Estimation')
plt.plot(odom_test['x'], odom_test['y'], 'r', lw=4, label='Test')

plt.xlabel('x [m]')
plt.ylabel('y [m]')
plt.legend(loc='upper right')
BoxOff()

Optimize odometry parameters

In [ ]:
# theta_opt, _ = EstimateOdometry_rLW(odom_est, x0_est, theta0)
theta_opt, _ = EstimateOdometry_all(odom_est, x0_est, theta0)
# theta_opt, _ = EstimateOdometry_crW(odom_est, x0_est, theta0)

print('theta0   : %s' % theta0)
print('theta_opt: %s' % theta_opt)

In [ ]:
# odom_test = odom_est
# x0_test = x0_est
odo_hat_0, loss_0 = EvalOdometry(odom_test, x0_test, *theta0)
odo_hat_opt, loss_opt = EvalOdometry(odom_test, x0_test, *theta_opt)

fig = plt.figure(31, clear=True)
plt.plot(d['pos']['x'], d['pos']['y'], label='')
plt.plot(odom_est['x'], odom_est['y'], 'b', lw=4, label='est')
plt.plot(odom_test['x'], odom_test['y'], 'r', lw=4, label='test')
plt.plot(odo_hat_0['x'], odo_hat_0['y'], 'm', lw=2, label='theta_0')
plt.plot(odo_hat_opt['x'], odo_hat_opt['y'], 'k', lw=2, label='theta_opt')

plt.xlabel('x [m]')
plt.ylabel('y [m]')
plt.legend()
plt.title('Dataset slow: Loss_opt/Loss_0 = %.1f [%%]' % (loss_opt/loss_0*100))
BoxOff()

## Fast speed

In [ ]:
d = data_dt['fast']
est = [6, 13]
test = [15, 20]
#test = est

odom_est, x0_est = GetCalibrationSection(d['pos'], d['odo'], est[0], est[1])
odom_test, x0_test = GetCalibrationSection(d['pos'], d['odo'], test[0], test[1])

gamma = 0.6
plt.figure(40, clear=True)
plt.plot(d['pos']['x'], d['pos']['y'], label='')
plt.plot(x0_est[0] + gamma*np.array([0, np.cos(x0_est[2])]),
         x0_est[1] + gamma*np.array([0, np.sin(x0_est[2])]), 'k', lw=2)
plt.plot(x0_test[0] + gamma*np.array([0, np.cos(x0_test[2])]),
         x0_test[1] + gamma*np.array([0, np.sin(x0_test[2])]), 'k', lw=2)

plt.plot(odom_est['x'], odom_est['y'], 'b', lw=4, label='Estimation')
plt.plot(odom_test['x'], odom_test['y'], 'r', lw=4, label='Test')

plt.xlabel('x [m]')
plt.ylabel('y [m]')
plt.legend(loc='upper right')
BoxOff()

Optimize odometry parameters

In [ ]:
# theta_opt, _ = EstimateOdometry_rLW(odom_est, x0_est, theta0)
theta_opt, _ = EstimateOdometry_all(odom_est, x0_est, theta0)
# theta_opt, _ = EstimateOdometry_crW(odom_est, x0_est, theta0)

print('theta0   : %s' % theta0)
print('theta_opt: %s' % theta_opt)

Plot results

In [ ]:
#odom_test = odom_est
#x0_test = x0_est

odo_hat_0, loss_0 = EvalOdometry(odom_test, x0_test, *theta0)
odo_hat_opt, loss_opt = EvalOdometry(odom_test, x0_test, *theta_opt)

fig = plt.figure(41, clear=True)
plt.plot(d['pos']['x'], d['pos']['y'], label='')
plt.plot(odom_est['x'], odom_est['y'], 'b', lw=4, label='est')
plt.plot(odom_test['x'], odom_test['y'], 'r', lw=4, label='test')
plt.plot(odo_hat_0['x'], odo_hat_0['y'], 'm', lw=2, label='theta_0')
plt.plot(odo_hat_opt['x'], odo_hat_opt['y'], 'k', lw=2, label='theta_opt')

plt.xlabel('x [m]')
plt.ylabel('y [m]')
plt.legend()
plt.title('Dataset fast: Loss_opt/Loss_0 = %.1f [%%]' % (loss_opt/loss_0*100))
BoxOff()